#Environment Setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive')



#Put Data in DataFrame (Articles)

In [ ]:
import pandas as pd
path = "PATH/TO/DATASETS" # Place dataset path here
import glob, os

dfFalse = pd.read_csv(path+"NewsFakeCOVID-19.csv", usecols=['title'])
dfFalse['label']=0
dfFalseJuly = pd.read_csv(path+"NewsFakeCOVID-19-JULY.csv", usecols=['title'])
dfFalseJuly['label']=0
dfTrue = pd.read_csv(path+"NewsRealCOVID-19.csv", usecols=['title'], nrows=len(dfFalse.values))
dfTrue['label']=1
dfTrueJuly = pd.read_csv(path+"NewsRealCOVID-19-JULY.csv", usecols=['title'], nrows=len(dfFalseJuly.values))
dfTrueJuly['label']=1

dfTotal = pd.concat([dfTrue, dfFalse, dfTrueJuly, dfFalseJuly])

X = dfTotal['title'].values
y = dfTotal['label'].values
print("len(X)", len(X))
print("len(y)", len(y))

#Next Steps With Data


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from keras.preprocessing.text import one_hot
from keras.preprocessing.text import hashing_trick

from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.optimizers import Adam
from keras import layers
from keras.layers import Activation
from keras import regularizers
from keras.regularizers import l1, l2, l1_l2
import math



X_train, X_test, y_train, y_test = train_test_split(
  X, y, test_size=0.25, random_state=450)

X_train = [hashing_trick(elem, 10000, hash_function='md5') for elem in X_train]
X_test =  [hashing_trick(elem, 10000, hash_function='md5') for elem in X_test]

X_train = pad_sequences(X_train, padding='post', maxlen=500)
X_test = pad_sequences(X_test, padding='post', maxlen=500)

print("Length of X_train:", len(X_train))
print("Length of X_test:", len(X_test))
print("Length of y_train:", len(y_train))
print("Length of y_test:", len(y_test))

model = Sequential()
model.add(layers.Embedding(10000, 64, input_length=500))
model.add(layers.Bidirectional(layers.LSTM(64, kernel_regularizer=l1_l2(l1=1e-5, l2=1e-4), bias_regularizer=l2(1e-4), recurrent_regularizer=l2(1e-5))))
model.add(layers.Dropout(0.7))
model.add(layers.Dense(1, activation='sigmoid'))

slowAdam = Adam(learning_rate=0.0001)
model.compile(loss='binary_crossentropy',
              optimizer=slowAdam,
              metrics=['accuracy'])
model.fit(X_train, y_train,
          epochs=5,
          validation_data=(X_test, y_test),
          batch_size=1)
model.summary()

model.save('PATH/TO/MODELS/CoAID_using_HashingTrick.h5') #Place intended path here


modelPrediction = model.predict(X_test)
print(len(modelPrediction))
modelPrediction = [math.floor(0.5+pred) for pred in modelPrediction] 


print(classification_report(y_test, modelPrediction))
print(accuracy_score(modelPrediction, y_test))
